In [2]:
import cv2
from gaze_tracking import GazeTracking

gaze = GazeTracking()
webcam = cv2.VideoCapture(0)

# Variables to store eye movement data
eye_movements = []
overall_movement = [(0, 0), (0, 0)]  # Separate overall movement for each eye
blink_count = 0  # Variable to store the number of blinks

# Variables for scrolling display
display_scroll_speed = 30  # Number of pixels to move the display text vertically every second
display_position = 20  # Initial y-coordinate for the display text
display_update_interval = 1  # Update display text position every second
last_display_update = 0  # Timestamp of the last display text update

while True:
    _, frame = webcam.read()
    gaze.refresh(frame)

    new_frame = gaze.annotated_frame()
    text = ""

    if gaze.is_right():
        text = "Looking to the left"
        eye_movements.append((0, -1))  # Add eye movement data to the list
    elif gaze.is_left():
        text = "Looking to the right"
        eye_movements.append((0, 1))  # Add eye movement data to the list
    elif gaze.is_center():
        text = "Looking straight ahead"
        eye_movements.append((0, 0))  # Add eye movement data to the list
    elif gaze.is_blinking():
        text = "Blinking"
        blink_count += 1

    # Update overall eye movement for each eye
    overall_movement[0] = tuple(map(sum, zip(overall_movement[0], eye_movements[-1])))
    overall_movement[1] = tuple(map(sum, zip(overall_movement[1], eye_movements[-1])))

    # Display overall movement for each eye in the upper right-hand corner
    overall_text = f"Overall: Left Eye - X: {overall_movement[0][0]}, Y: {overall_movement[0][1]} " \
                   f"Right Eye - X: {overall_movement[1][0]}, Y: {overall_movement[1][1]}"
    cv2.putText(new_frame, text, (60, 60), cv2.FONT_HERSHEY_COMPLEX, 2, (255, 0, 0), 2)

    # Scroll the display text vertically every second
    current_time = cv2.getTickCount() / cv2.getTickFrequency()
    if current_time - last_display_update >= display_update_interval:
        last_display_update = current_time
        display_position += display_scroll_speed

    # Display the scrolling eye movement data
    num_movements = len(eye_movements)
    if num_movements > 0:
        latest_movement = eye_movements[-1]
        display_text = f"X: {latest_movement[0]}, Y: {latest_movement[1]}"
        cv2.putText(new_frame, display_text, (20, display_position), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 1)

        # Reset display position if it reaches the bottom of the screen
        if display_position > new_frame.shape[0] - 30:
            display_position = 20

    # Display the overall movement for each eye in the upper right-hand corner
    cv2.putText(new_frame, overall_text, (new_frame.shape[1] - 450, 60), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255), 1)

    # Display the number of blinks in the lower right-hand corner
    blink_text = f"Blink Count: {blink_count}"
    cv2.putText(new_frame, blink_text, (new_frame.shape[1] - 250, new_frame.shape[0] - 30),
                cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255), 1)

    cv2.imshow("Demo", new_frame)

    if cv2.waitKey(1) == 27:
        break
